Importing python libraries

In [15]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

Computes the MSE of two images

    Args:
        array1 ([numpy.ndarray]): First image
        array2 ([numpy.ndarray]): Seccond image
    Returns:
        float : MSE error

In [16]:
def mse(array1 , array2):
    array1=array1.flatten()
    array2=array2.flatten()
    return np.sqrt(((array1 - array2)**2).mean())

Computes and generate the gradient of a pixel at the coordinates (x,y) of the image img

    Args:
        img ([numpy.ndarray]): Array of the image
        x ([int]): x position
        y ([int]): y position

    Returns:
        float : magnitude of the gradient
        direction : [direction of the gradient


In [17]:
def gradiant(img,x,y):
    gx=img[y+1][x]-img[y-1][x]
    gy=img[x+1][y]-img[x-1][y]
    magnitude = np.sqrt(gx**2+gy**2)
    direction = np.sign(gy) * 90 if gx==0 else np.degrees(np.arctan(gy/gx))
    return magnitude , direction

Converts an img to LBP

    Args:
        img ([numpy.ndarray]): Source image

    Returns:
        [numpy.ndarray] : LBP filter image

In [18]:
def lbp(img):

    img = np.pad(img, 1, mode="edge")
    new_img = np.zeros(img.shape)
    for j in range(1, new_img.shape[0]-1):
        for i in range(1,  new_img.shape[1]-1):
            vector = []
            center_value=img[j][i]
            vector.append(0 if img[j-1][i+1] >= center_value else 1)
            vector.append(0 if img[j][i+1] >= center_value else 1)
            vector.append(0 if img[j+1][i+1] >= center_value else 1)
            vector.append(0 if img[j+1][i] >= center_value else 1)
            vector.append(0 if img[j+1][i-1] >= center_value else 1)
            vector.append(0 if img[j][i-1] >= center_value else 1)
            vector.append(0 if img[j-1][i-1] >= center_value else 1)
            vector.append(0 if img[j-1][i] >= center_value else 1)
            new_img[j][i] = np.sum(np.multiply(vector, [1, 2, 4, 8, 16, 32, 64, 128]))
    return new_img[1:new_img.shape[0]-1:1,1:new_img.shape[1]-1]


Apply hog to an img and extracts the direction

    Args:
        image ([type]): [description]

    Returns:
        [type]: [description]

In [19]:
def generateDirections(image):
    direction=[]
    image = np.pad(image, 1, mode="edge")
    for j in range(1, image.shape[0]-1):
        for i in range(1,  image.shape[1]-1):
            direction.append(gradiant(image,j,i)[1])
    return direction


Creates an image's histogram with LBP filter

    Args:
        image (numpy.ndarray): Source image

    Returns:
        numpy.ndarray: Histogram of the source image
    """

In [20]:
def getHist(image):
    toret=np.zeros(256)
    unique=np.unique(image , return_counts=True)
    for i in range(0,len(unique[0])):
        toret[int(unique[0][i])] = unique[1][i]
    return toret

Generates HOG's array from another array , i choosed to divide it into 8 possible directions (from 0 to 180) with an offset of 90 , each
class has a span of 22.5

    Args:
        array ([numpy.ndarray]): input array

    Returns:
        [numpy.ndarray]: histogram of the input array 
    """

In [21]:
def getHog(array):

    toret = np.zeros(9)
    for i in array:
        index=int((i+90)/22.5)
        toret[index]+=1
    return toret

returns the histograms of an image after dividing it in sizexsize blocks and applying lbp filter

    Args:
        image ('numpy.ndarray'): Source image
        size ('int'): size of the block

    Returns:
        'numpy.ndarray': the histograms of sizexsize divided image

In [22]:
def divBloc(image,size):
    histograms = []
    hog=[]
    image = np.pad(image, 1, mode="edge")
    for j in range(1,image.shape[0]-1,size):
        for i in range(1,image.shape[1]-1,size):
            crop = image[j:j+size,i:i+size]
            crop_lbp = lbp(crop)
            histograms.append(getHist(crop_lbp))
    return np.array(histograms)

returns the histograms of an image after dividing it in sizexsize blocks and applying hog

    Args:
        image ('numpy.ndarray'): Source image
        size ('int'): size of the block

    Returns:
        'numpy.ndarray': the histograms of sizexsize divided image

In [23]:
def divBlochog(image,size):
    hog=[]
    image = np.pad(image, 1, mode="edge")
    for j in range(1,image.shape[0]-1,size):
        for i in range(1,image.shape[1]-1,size):
            crop = image[j:j+size,i:i+size]
            crop_hog = generateDirections(crop)
            hog.append(getHog(crop_hog))
    return  np.array(hog)

Main code

Load the haar features cascade

In [24]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

Loading the imagges from the dataset and applyting LBP and HOG

In [25]:
bloc_size=8
num_images=27
lbp_reference_hist=[]
hog_reference_hist=[]
for i in tqdm(range(1,num_images+1)):
    try :
        temp=cv2.imread('src\\'+str(i)+'.jpg')
        temp=cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(temp, 1.1, 4)
        for (x, y, w, h) in faces:
            temp=cv2.resize(temp[y:y+h,x:x+w],(64,128))
            temp_lbp=np.copy(temp)
            lbp_reference_hist.append(divBloc(temp_lbp,bloc_size))
            hog_reference_hist.append(divBlochog(temp,bloc_size))
    except :
        print(i)

  0%|          | 0/27 [00:00<?, ?it/s]ipykernel_launcher:2: RuntimeWarning: overflow encountered in ubyte_scalars
ipykernel_launcher:3: RuntimeWarning: overflow encountered in ubyte_scalars
100%|██████████| 27/27 [00:15<00:00,  1.76it/s]


Capture video from webcam

In [26]:
cap = cv2.VideoCapture(0)

Main loop

In [27]:
while True:
    _, img = cap.read() #Read the image given by the cam and puts it in img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converts img to grayscale
    faces = face_cascade.detectMultiScale(gray, 1.1, 4) #apllies hog features and detects al lfaces in the image
    facesResized =[]
    for (x, y, w, h) in faces: #for each face in the image
        img_resized=cv2.resize(gray[y:y+h,x:x+w],(64,128)) # Exctracts the face and resize it into 64x128 format
        lbp_image_hist = divBloc(img_resized,bloc_size) # apply lbp to the resized face
        hog_image_hist=divBlochog(img_resized,bloc_size) # apply hog to the resized face

        error=0
        for i in range (0,len(lbp_reference_hist)): # for each dataset image
            # computes mse between the current image and the image i from the dataset and summs the error
            error+=(mse(lbp_image_hist,lbp_reference_hist[i])) 
            error+=(mse(hog_image_hist,hog_reference_hist[i]))
        color = (0, 0, 255)
        if error < 157 : # if the total sum is < to a certain treshhold , considers the face in the dataset as BANDOUI Nazim
            color = (0, 255, 0)
        print(error)
        # Draw the rectangle around each face
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
    try :
        cv2.imshow('BANDOUI NAZIIM - PROJET ATELIER CREATIF', img)
    except :
        pass
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

ipykernel_launcher:2: RuntimeWarning: overflow encountered in ubyte_scalars
ipykernel_launcher:3: RuntimeWarning: overflow encountered in ubyte_scalars
156.82501973164773
147.70636249158864
163.18641194643695
129.08999401850915
159.72676167398689
123.92708319084049
122.98363302746372
125.56800147822646
123.00804285794281
125.49169535589405
123.70955944267922
126.46790456705432
124.24957138201643
123.70330681946805
128.62923425318155
153.9146511790062
160.71362591826644
157.73280322209823
160.58930013751694
161.29683381705314
158.34008565429858
145.13080464207084
162.43774424990406
141.65064038247385
164.49124804329207
144.84869026106495
161.8344525449283
145.58708210159756
144.12857106488318
140.0998851288159
146.6728360895877
143.91768821338803
161.26163776697075
146.7654104463857
143.6101272542066
140.2994305563453
143.92549664060465
149.02709314733528
163.45012437770984
146.34580281420128
162.97799593422008
145.39672155506486
145.60076936007277
144.99749152604602
146.32254783287635


release the videocapture object

In [28]:
cap.release()
cv2.destroyAllWindows()